# Introduction

This notebook analyzes the results of an "observer" agent group and a "target" agent group from a simulation and produces plots to visualize the revisit and data handling behavior between them. To understand the required user inputs, read the [Analysis Configuration](#analysis-configuration) section.

## Contents

This notebook produces the following plots:

- Revisit
  - [Revisit Timeline](#revisit-timeline)
  - [Revisit Duration Distribution](#revisit-duration-distribution)
- Data Transmit
  - [Transmit Bit Rate by Agent](#transmit-bit-rate-by-agent)
  - [Transmit Bit Rate by Target](#transmit-bit-rate-by-target)
  - [Transmit Bit Rate by Data Type](#transmit-bit-rate-by-data-type)
  - [Cumulative Data Transmitted by Agent](#cumulative-data-transmitted-by-agent)
  - [Cumulative Data Transmitted by Target](#cumulative-data-transmitted-by-target)
  - [Cumulative Data Transmitted by Data Type](#cumulative-data-transmitted-by-data-type)
  - [Cumulative Data Transmitted to All Targets](#cumulative-data-transmitted-to-all-targets)
- Data Receive
  - [Receive Bit Rate by Agent](#receive-bit-rate-by-agent)
  - [Receive Bit Rate by Target](#receive-bit-rate-by-target)
  - [Receive Bit Rate by Data Type](#receive-bit-rate-by-data-type)
  - [Cumulative Data Received by Agent](#cumulative-data-received-by-agent)
  - [Cumulative Data Received by Target](#cumulative-data-received-by-target)
  - [Cumulative Data Received by Data Type](#cumulative-data-received-by-data-type)
  - [Cumulative Data Received to All Targets](#cumulative-data-received-from-all-targets)
- Data Storage
  - [Total Data Storage Usage by Agent](#total-data-storage-usage-by-agent)
  - [Total Data Storage Usage by all Agents](#total-data-storage-usage-by-all-agents)
  - [Data Storage Usage by Data Type](#data-storage-usage-by-data-type)
  - [Data Minimum, Maximum, and Average Age by Agent](#data-minimum-maximum-and-average-age-by-agent)

# Setup

#### Package Installation and Import

The following cell should install all necessary packages for this notebook.

In [ ]:
%pip install pandas plotly IPython ipywidgets nbformat
%pip install "sedaro>=4.17.3"

In [97]:
import json
from collections import defaultdict

import pandas as pd
import plotly.express as px
from data_analysis import target_data_results
from IPython.display import HTML, display
from revisit_analysis import target_revisit_results, target_revisit_statistics
from sedaro import SedaroApiClient
from sedaro.modsim import data_scale_unit

#### Important: Read Before Running

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the cloned `modsim-notebooks` directory with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [98]:
with open('../../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../../config.json', 'r') as file:
    config = json.load(file)

HOST = config['HOST']  # Sedaro instance URL

sedaro = SedaroApiClient(API_KEY, HOST)

### Analysis Configuration

Below are fields that point the notebook to simulation data and drive analysis behavior.
- `SCENARIO_BRANCH_ID`: The branch ID of the scenario that you would like to analyze.
- `JOB_ID`: An optional field for the ID of the specific simulation job that you would like to analyze. If left as an empty string (`""`), the most recent simulation from your Scenario will be analyzed.
- `OBSERVER_AGENT_GROUP_ID`: The ID of the agent group that contains the agents that revisited, transmitted data to, and/or received data from the agents in the target agent group.
- `TARGET_AGENT_GROUP_ID`: The ID of the target agent group that contains the agents that were revisited by, transmitted data to, and/or received data from the agents in the observer agent group.
- `REVISIT_ANALYSIS`: Whether to perform revisit analysis from the observer agent group to the target agent group.
- `REVISIT_ANALYSIS_CONDITION`: The name of the condition that is met when an observer agent revisits a target agent. There must be a `TargetGroupCondition` with this name on each of the observer agent templates.
- `DATA_ANALYSIS`: Whether to perform data analysis for the observer agent group and between the observer agent group and the target agent group.
- `DATA_ANALYSIS_DATA_TYPES`: The names of the data types that should be considered for data analysis. Data types that are not in this list will not affect data analysis results. If the list is empty, all data types will be considered.
- `px.defaults.template`: Set to `"plotly_dark"` for dark themed plots or `"plotly_white"` for light themed plots.

The fields are already filled with values that point towards [the Wildfire demo in the Sedaro Shared Demos workspace](https://satellite.sedaro.com/projects/PKCbxBlcdpFkvHxfKKRKqT). These will allow you to run this notebook without further configuration if you have an account with a valid API key.

In [99]:
SCENARIO_BRANCH_ID: str = "PKCbxCXvQhCn8glwd5mkSj"  # ID of the Scenario branch
JOB_ID: str = ""  # optional ID of the simulation job

OBSERVER_AGENT_GROUP_ID: str = "NT08_XVROe2Va_eBgCK2k"  # ID of the observer AgentGroup
TARGET_AGENT_GROUP_ID: str = "NTKAJurTrHOKg99QP6Hpk"  # ID of the target AgentGroup


REVISIT_ANALYSIS: bool = True  # if True, the revisit analysis will be performed
REVISIT_ANALYSIS_CONDITION: str = "Ground Station Elevation Angle"  # name of the condition that indicates a revisit

DATA_ANALYSIS: bool = True  # if True, the data analysis will be performed
DATA_ANALYSIS_DATA_TYPES: list[str] = []  # names of the data types to include in the analysis, empty list includes all

px.defaults.template = "plotly_dark"
# px.defaults.template = "plotly_white"

# Analysis

## Download Results

In [100]:
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

observer_agents = list(scenario.AgentGroup.get(OBSERVER_AGENT_GROUP_ID).agentAssociations.keys())
target_agents = list(scenario.AgentGroup.get(TARGET_AGENT_GROUP_ID).agentAssociations.keys())
# ^^^ this will get us the names for targets created during build to populate a TG, but not for targets which exist on
# the model
observer_to_target_mapping = defaultdict(dict)
for observer_agent in observer_agents:
    for agent_data, mapping_data in observer_agent.targetMapping.items():
        for target_id in mapping_data['associatedTargets']:
            observer_to_target_mapping[observer_agent.name][target_id] = agent_data.id
observer_to_target_mapping = dict(observer_to_target_mapping) # default behavior ok for population, missing key should error below

In [ ]:
results = scenario.simulation.results(JOB_ID)

observer_results = {agent.name: results.agent(agent.name) for agent in observer_agents}

results.summarize()

## Perform Analysis

In [104]:
if REVISIT_ANALYSIS or DATA_ANALYSIS:
    observer_template_branches = {template_ref: sedaro.agent_template(template_ref)
                                  for template_ref in set(agent.templateRef for agent in observer_agents)}
    observer_agent_templates = {agent.name: observer_template_branches[agent.templateRef]
                                for agent in observer_agents}

if REVISIT_ANALYSIS:
    revisit_condition_ids = {}
    for agent_name, agent_template in observer_agent_templates.items():
        revisit_condition_id = next((condition.id for condition in agent_template.TargetGroupCondition.get_all()
                                     if condition.name == REVISIT_ANALYSIS_CONDITION), None)
        if revisit_condition_id is None:
            raise ValueError(
                f"The revisit analysis condition ({REVISIT_ANALYSIS_CONDITION}) is not present in the agent template for {agent_name}.")
        revisit_condition_ids[agent_name] = revisit_condition_id
    target_revisits = target_revisit_results(observer_results, revisit_condition_ids,
                                             {target.id: target.name for target in target_agents}, observer_to_target_mapping)
    revisit_statistics = target_revisit_statistics(target_revisits)

if DATA_ANALYSIS:
    transmit_data_results, receive_data_results, data_storage_results = target_data_results(
        observer_agent_templates, observer_results, {target.id: target.name for target in target_agents}, DATA_ANALYSIS_DATA_TYPES)

# Results

## Revisit

#### Revisit Timeline

In [ ]:
if REVISIT_ANALYSIS:
    fig = px.timeline(target_revisits, x_start="Start", x_end="End",
                      y="Target", color="Agent", title="Revisit Timeline")
    fig.show()
    display(HTML(revisit_statistics.to_html(index=False)))
else:
    print("No revisit analysis performed.")

#### Revisit Duration Distribution

In [ ]:
if REVISIT_ANALYSIS:
    revisit_distribution_results = target_revisits.copy(deep=True)
    # convert durations to datetime for plotly formatting
    revisit_distribution_results.Duration = revisit_distribution_results.Duration + pd.to_datetime("1970-01-01")
    fig = px.violin(revisit_distribution_results, x="Target", y="Duration",
                    box=True, title="Revisit Duration Distribution")
    fig.update_yaxes(tickformat="%M:%S", title="Duration (MM:SS)")
    fig.show()
else:
    print("No revisit analysis performed.")

## Data Transmit Interfaces

### Transmit Bit Rate

#### Transmit Bit Rate by Agent

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.groupby(["Time", "Agent"])["Bit Rate"].sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Agent", title="Transmit Bit Rate by Agent", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

#### Transmit Bit Rate by Target

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.groupby(["Time", "Target"])["Bit Rate"].sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Target", title="Transmit Bit Rate by Target", line_shape="vh")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".2s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

#### Transmit Bit Rate by Data Type

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.groupby(["Time", "Data Type"]).sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Data Type",
                      title="Transmit Bit Rate by Data Type", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".2s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

### Cumulative Data Transmitted

#### Cumulative Data Transmitted by Agent

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.sort_values(by="Time")
        df["Cumulative Data Transmitted"] = df.groupby("Agent")["Data Transmitted"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Transmitted", color="Agent",
                      title="Cumulative Data Transmitted by Agent", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Transmitted"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Transmitted ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Transmitted by Target

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.sort_values(by="Time")
        df["Cumulative Data Transmitted"] = df.groupby("Target")["Data Transmitted"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Transmitted", color="Target",
                      title="Cumulative Data Transmitted by Target", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Transmitted"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Transmitted ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Transmitted by Data Type

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.sort_values(by="Time")
        df["Cumulative Data Transmitted"] = df.groupby("Data Type")["Data Transmitted"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Transmitted",
                      color="Data Type", title="Cumulative Data Transmitted by Data Type", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Transmitted"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Transmitted ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Transmitted to All Targets

In [ ]:
if DATA_ANALYSIS:
    if not transmit_data_results.empty:
        df = transmit_data_results.sort_values(by="Time")
        df["Cumulative Data Transmitted"] = df["Data Transmitted"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Transmitted",
                      title="Cumulative Data Transmitted to All Targets", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Transmitted"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale, fill='tozeroy'))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Transmitted ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No transmit data.")
else:
    print("No data analysis performed.")

## Data Receive Interfaces

### Receive Bit Rate

#### Receive Bit Rate by Agent

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.groupby(["Time", "Agent"])["Bit Rate"].sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Agent", title="Receive Bit Rate by Agent", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

#### Receive Bit Rate by Target

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.groupby(["Time", "Target"])["Bit Rate"].sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Target", title="Receive Bit Rate by Target", line_shape="vh")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".2s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

#### Receive Bit Rate by Data Type

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.groupby(["Time", "Data Type"]).sum().reset_index()
        fig = px.line(df, x="Time", y="Bit Rate", color="Data Type",
                      title="Receive Bit Rate by Data Type", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Bit Rate"].max())
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Bit Rate ({unit_prefix}bps)"))
        fig.update_yaxes(tickformat=".2s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

### Cumulative Data Received

#### Cumulative Data Received by Agent

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.sort_values(by="Time")
        df["Cumulative Data Received"] = df.groupby("Agent")["Data Received"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Received", color="Agent",
                      title="Cumulative Data Received by Agent", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Received"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Received ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Received by Target

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.sort_values(by="Time")
        df["Cumulative Data Received"] = df.groupby("Target")["Data Received"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Received", color="Target",
                      title="Cumulative Data Received by Target", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Received"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Received ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Received by Data Type

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.sort_values(by="Time")
        df["Cumulative Data Received"] = df.groupby("Data Type")["Data Received"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Received",
                      color="Data Type", title="Cumulative Data Received by Data Type", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Received"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Received ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

#### Cumulative Data Received from All Targets

In [ ]:
if DATA_ANALYSIS:
    if not receive_data_results.empty:
        df = receive_data_results.sort_values(by="Time")
        df["Cumulative Data Received"] = df["Data Received"].cumsum()
        fig = px.line(df, x="Time", y="Cumulative Data Received",
                      title="Cumulative Data Received to All Targets", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Cumulative Data Received"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale, fill='tozeroy'))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Cumulative Data Received ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No receive data.")
else:
    print("No data analysis performed.")

## Data Storage

#### Total Data Storage Usage by Agent

In [ ]:
if DATA_ANALYSIS:
    if not data_storage_results.empty:
        df = pd.DataFrame(data_storage_results.groupby(["Time", "Agent"])["Usage"].sum().reset_index())
        fig = px.line(df, x="Time", y="Usage", color="Agent",
                      title="Total Data Storage Usage by Agent", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Usage"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Usage ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No data storage data.")
else:
    print("No data analysis performed.")

#### Total Data Storage Usage by All Agents

In [ ]:
if DATA_ANALYSIS:
    if not data_storage_results.empty:
        df = pd.DataFrame(data_storage_results.groupby("Time")["Usage"].sum()).reset_index()
        df.rename(columns={"Usage": "Total Usage"}, inplace=True)
        fig = px.line(df, x="Time", y="Total Usage", title="Total Data Storage Usage by All Agents", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Total Usage"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale, fill='tozeroy'))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Total Usage ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No data storage data.")
else:
    print("No data analysis performed.")

#### Data Storage Usage by Data Type

In [ ]:
if DATA_ANALYSIS:
    if not data_storage_results.empty:
        df = pd.DataFrame(data_storage_results.groupby(["Time", "Data Type"])["Usage"].sum()).reset_index()
        fig = px.line(df, x="Time", y="Usage", color="Data Type",
                      title="Total Data Storage Usage by Data Type", line_shape="hv")
        scale, unit_prefix = data_scale_unit(df["Usage"].max(), bytes=True)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / scale))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Usage ({unit_prefix}B)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No data storage data.")
else:
    print("No data analysis performed.")

#### Data Minimum, Maximum, and Average Age by Agent

In [ ]:
if DATA_ANALYSIS:
    if not data_storage_results.empty:
        df = pd.DataFrame()
        df["Time"] = data_storage_results.groupby(["Time", "Agent"]).sum().reset_index()["Time"]
        df["Min. Age"] = data_storage_results.groupby(["Time", "Agent"])["Min. Age"].min().reset_index()["Min. Age"]
        df["Max. Age"] = data_storage_results.groupby(["Time", "Agent"])["Max. Age"].max().reset_index()["Max. Age"]
        df["Average Age"] = data_storage_results.groupby(["Time", "Agent"]).apply(
            lambda x: (x["Usage"] * x["Average Age"]).sum() / x["Usage"].sum() if x["Usage"].sum() != 0 else 0
        ).reset_index(level=[0, 1], drop=True)
        df["Agent"] = data_storage_results.groupby(["Time", "Agent"]).first().reset_index()["Agent"]
        fig = px.line(df, x="Time", y=["Min. Age", "Max. Age"],
                      color="Agent", title="Data Minimum, Maximum, and Average Age by Agent", line_shape="hv")
        fig.for_each_trace(lambda trace: trace.update(fill='tonexty', opacity=0.5,
                                                      line={"dash": "dot"}, showlegend=False))
        fig.add_traces(px.line(df, x="Time", y="Average Age", color="Agent").data)
        fig.for_each_trace(lambda trace: trace.update(y=trace.y / 60))
        fig.for_each_yaxis(lambda axis: axis.update(title=f"Age (min)"))
        fig.update_yaxes(tickformat=".3s")
        fig.show()
    else:
        print("No data storage data.")
else:
    print("No data analysis performed.")